In [ ]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [ ]:
import yaml
import pandas as pd
import datetime 
import gc
import re

%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - SummonsNumber
    - PlateID
    - RegistrationState
    - PlateType
    - IssueDate
    - ViolationCode 
    - VehicleBodyType
    - VehicleMake
    - IssuingAgency
    - StreetCode1
    - StreetCode2
    - StreetCode3
    - VehicleExpirationDate
    - ViolationLocation
    - ViolationPrecinct
    - IssuerPrecinct
    - IssuerCode
    - IssuerCommand
    - IssuerSquad
    - ViolationTime
    - TimeFirstObserved
    - ViolationCounty
    - ViolationInFrontOfOrOpposite
    - HouseNumber
    - StreetName
    - IntersectingStreet
    - DateFirstObserved
    - LawSection
    - SubDivision
    - ViolationLegalCode
    - DaysParkingInEffect
    - FromHoursInEffect
    - ToHoursInEffect
    - VehicleColor-
    - UnregisteredVehicle?
    - VehicleYear 
    - MeterNumber
    - FeetFromCurb
    - ViolationPostCode
    - ViolationDescription
    - NoStandingorStoppingViolation
    - HydrantViolation
    - DoubleParkingViolation
    - Latitude
    - Longitude
    - CommunityBoard
    - CommunityCouncil
    - CensusTract
    - BIN
    - BBL
    - NTA

In [ ]:
import utility as util
config_data = util.read_config_file("store.yaml")

In [ ]:
config_data

In [ ]:
from dask import dataframe as dd
df_sample = dd.read_csv('Parking_Violations.csv',delimiter=',')
df_sample.head()

In [ ]:
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'

In [ ]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

In [ ]:
util.col_header_val(df,config_data)

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

In [ ]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('Parking_Violations.csv',delimiter=',')

df.to_csv('Parking_Violations.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

In [ ]:
import os
entries = os.listdir('Parking_Violations.csv.gz/')
for entry in entries:
    print(entry)

In [ ]:
os.path.getsize('Parking_Violations.csv.gz')